# 🛠️ ETL (Extract, Transform, Load)
- **Extraccion de datos**:  Leer y cargar los datos desde las fuentes de datos (archivos, bases de datos, etc.).

- **Transformacion de datos**: Realizar las transformaciones necesarias en los datos, como: 
  - Renombrar columnas para mantener consistencia.
  - Convertir tipos de datos.
  - Eliminar columnas no relevantes o con demasiados valores faltantes.
  - Aplicar cualquier otra transformación requerida (cálculos, operaciones, etc.).
- **Carga de datos**: Combinar los conjuntos de datos transformados (si es necesario) y cargarlos en una estructura de datos adecuada (DataFrame, base de datos, etc.) para su posterior análisis.

Importamos los modulos que vamos a usar

In [1]:
from data_utils import data_type_check,data_type_check_EDA
import re
import numpy as np 
import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer

### 📦 Extraccion de los datos y primera exploración


In [2]:
homicidios_hechos = pd.read_excel("../0 Dataset/homicidios.xlsx", sheet_name="HECHOS")
homicidios_victimas = pd.read_excel("../0 Dataset/homicidios.xlsx", sheet_name="VICTIMAS")
comunas = pd.read_excel("../0 Dataset/comunas.xlsx")

___

# 🔁 TRANSFORM Dataset homicidios_hechos

In [3]:
print("Veamos 3 muestras del Dataset: ")
homicidios_hechos.sample(3)

Veamos 3 muestras del Dataset: 


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
270,2017-0157,1,2017-12-17,2017,12,17,09:02:00,9,AV. GRAL. PAZ Y HUMAITA,GRAL PAZ,...,NaN,HUMAITA,"PAZ, GRAL. AV. y HUMAITA",9,Point (94007.62506099 98145.35456663),-58.52866211,-34.64597065,AUTO-PASAJEROS,AUTO,PASAJEROS
659,2021-0061,1,2021-07-24,2021,7,24,09:15:00,9,"MORENO, PERITO AV. Y ALCORTA, AMANCIO AV.",AVENIDA,...,NaN,"ALCORTA, AMANCIO AV.","MORENO, PERITO AV. y ALCORTA, AMANCIO AV.",4,Point (105303.77662691 97684.64886839),-58.40544588,-34.65012801,PEATON-CARGAS,PEATON,CARGAS
321,2018-0047,1,2018-05-08,2018,5,8,19:00:00,19,Av. Corrientes y Av. Dorrego,AVENIDA,...,NaN,DORREGO AV.,CORRIENTES AV. y DORREGO AV.,15,Point (101440.29935254 104167.40204334),-58.44759966,-34.59170240,PEATON-PASAJEROS,PEATON,PASAJEROS


In [4]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Veamos la descripcion estadistica con .describe()


In [5]:
homicidios_hechos.describe()

,N_VICTIMAS,FECHA,AAAA,MM,DD,Altura,COMUNA
count,696.000000,696,696.000000,696.000000,696.000000,129.000000,696.000000
mean,1.030172,2018-09-14 05:12:24.827586048,2018.188218,6.692529,15.936782,3336.635659,7.425287
min,1.000000,2016-01-01 00:00:00,2016.000000,1.000000,1.000000,30.000000,0.000000
25%,1.000000,2017-04-02 12:00:00,2017.000000,4.000000,9.000000,1359.000000,4.000000
50%,1.000000,2018-07-12 12:00:00,2018.000000,7.000000,16.000000,2551.000000,8.000000
75%,1.000000,2020-01-04 06:00:00,2020.000000,10.000000,23.000000,4500.000000,11.000000
max,3.000000,2021-12-30 00:00:00,2021.000000,12.000000,31.000000,16080.000000,15.000000
std,0.179393,NaN,1.683754,3.571309,8.639646,3060.641793,4.387050


### Nulos y duplicados
Usando una funcion personalizada data_type_check importada desde data_utils.py: 
- mostramos total de nulos, tipo de datos, dimensiones.

In [6]:
data_type_check(homicidios_hechos)


 Resumen del dataframe:

Dimensiones:  (696, 21)
                  columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0                      ID      100.00     0.00            0          object
1              N_VICTIMAS      100.00     0.00            0           int64
2                   FECHA      100.00     0.00            0  datetime64[ns]
3                    AAAA      100.00     0.00            0           int64
4                      MM      100.00     0.00            0           int64
5                      DD      100.00     0.00            0           int64
6                    HORA      100.00     0.00            0          object
7                      HH      100.00     0.00            0          object
8         LUGAR_DEL_HECHO      100.00     0.00            0          object
9           TIPO_DE_CALLE      100.00     0.00            0          object
10                  Calle       99.86     0.14            1          object
11                 Altura       18.53 

Revisamos los campos con nulos

In [7]:
homicidios_hechos[homicidios_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Encontramos que los nulos se encuentran distribuidos en cuatro columnas: Altura, Dirección Normalizada, Cruce y Calle.
* La columna Calle tiene solo un dato nulo.
* La columna Dirección Normalizada presenta 8 datos nulos.
* La columna Cruce tiene 171 datos nulos, lo que representa el 24.57% de los datos de la columna.
* La columna Altura contiene 567 datos nulos, equivalentes al 81.47% de la columna.

Debido a la gran cantidad de faltantes en Altura y la imposibilidad de completarlos, se decide eliminar esta columna. 
Altura se refiere a la numeración de la calle donde ocurre el siniestro, y en la mayoría de los casos, estos se producen en cruces o esquinas.

La columna Cruce solo contiene datos cuando el incidente ocurre en un cruce de calles, por lo que se mantiene tal cual.

Aunque la columna Dirección Normalizada no se puede completar con las demás columnas del dataset, tiene solo un 1.15% de datos nulos, por lo que se conserva.

In [8]:
# Se elimina la columna "Altura"
homicidios_hechos= homicidios_hechos.drop("Altura", axis=1)
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección Normalizada  688 non-null    object        
 13  COMUN

In [9]:
print("Cantidad de valores duplicados: ")
homicidios_hechos.duplicated().sum()

Cantidad de valores duplicados: 


0

No se encontraron datos duplicados

___

### Columna HH

La primer columna a transformar es HH, su tipo de dato es object y lo queremos en numerico.
- Primero analizamos los datos de la columna y luego la transformamos a numerico

In [10]:
homicidios_hechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

Entre los valores encontramos SD, que refiere a SIN DATO. Como no podemos ponerle un reemplazo inventado, elegimos dejarlo como está.

In [11]:
# Cambio a tipo de dato numerico y luego a entero 
homicidios_hechos["HH"] = pd.to_numeric(homicidios_hechos ["HH"], errors="coerce")
homicidios_hechos["HH"] = homicidios_hechos["HH"].astype("Int64")

___

### Columnas pos x pos y

Ahora vamos a limpiar los campos POS X y POS Y de posiciones no validas que se encuentran representadas con el valor de un punto **"."**

In [12]:
#Contar las columnas que tengan el valor . en pos x y pos y
homicidios_hechos.loc[(homicidios_hechos['pos x'] == '.') | (homicidios_hechos['pos y'] == '.')].shape


(12, 20)

Encontramos 12 campos de . en pos x y 20 en pos y

In [13]:
'''
Este código filtra las filas con posiciones inválidas
establece estas posiciones como NaN
y convierte las posiciones válidas de cadena a valores numéricos de tipo float
'''
# Filtrar filas con coordenadas no válidas
coordenadas_validas = ~(homicidios_hechos['pos x'].str.contains('[^0-9.,-]') | homicidios_hechos['pos y'].str.contains('[^0-9.,-]') | (homicidios_hechos['pos x'] == '.') | (homicidios_hechos['pos y'] == '.'))

# Establecer coordenadas no válidas en NaN
homicidios_hechos.loc[~coordenadas_validas, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].str.replace(',', '.').astype(float)
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].str.replace(',', '.').astype(float)
# Reemplazar NaN con 0
homicidios_hechos.fillna({'pos x': 0, 'pos y': 0}, inplace=True)



Tiene coordenadas que podemos ubicar en un mapa.

Vamos a extraerlas para poder utilizarlas

In [14]:
#Unir pos x + pos y, separados los dos pos una coma
homicidios_hechos['Coordenada'] = homicidios_hechos['pos x'].astype(str) + ',' + homicidios_hechos['pos y'].astype(str)

#Se observa el resultado
homicidios_hechos.sample(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,Coordenada
252,2017-0136,1,2017-11-16,2017,11,16,08:43:00,8,DEL BARCO CENTENERA Y RABANAL,AVENIDA,...,"RABANAL, FRANCISCO, INTENDENTE AV.","BARCO CENTENERA DEL y RABANAL, FRANCISCO, INTE...",4,Point (104127.10295709 97119.53980201),-58.418278,-34.655227,PEATON-PASAJEROS,PEATON,PASAJEROS,"-58.41827843,-34.65522746"
209,2017-0084,1,2017-07-05,2017,7,5,12:49:00,12,AV. DEL LIBERTADOR Y RODRIGUEZ PEÑA,AVENIDA,...,RODRIGUEZ PE?A,DEL LIBERTADOR AV. y RODRIGUEZ PEÃ‘A,2,Point (107180.90591568 104682.54945160),-58.385029,-34.587035,PEATON-PASAJEROS,PEATON,PASAJEROS,"-58.38502878,-34.58703485"


In [15]:
#Se observan las columnas resultantes
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'Coordenada'],
      dtype='object')

In [16]:
homicidios_hechos.sample(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,Coordenada
571,2020-0051,1,2020-11-05,2020,11,5,00:40:00,0,AVELLANEDA AV. Y MERCEDES,AVENIDA,...,MERCEDES,AVELLANEDA AV. y MERCEDES,10,Point (98015.81258802 99939.73589695),-58.484938,-34.629811,MOTO-AUTO,MOTO,AUTO,"-58.48493828,-34.62981124"
679,2021-0081,1,2021-11-02,2021,11,2,13:35:00,13,AU FRONDIZI SUBIDA BRANDSEN,AUTOPISTA,...,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y B...,4,Point (107979.30419282 99027.13636949),-58.376274,-34.638009,MOTO-CARGAS,MOTO,CARGAS,"-58.37627355,-34.63800916"


___

### Columna comuna

En la columna 'Comuna' tenemos una numeracion del 1 al 15. 
Esta numeración identifica a cada una de las divisiones comunales de la ciudad autónoma de Buenos Aires.



In [17]:
homicidios_hechos["COMUNA"].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5,  0],
      dtype=int64)

Vamos a agregar valor, agregaremos los Barrios que la conforman.

Para eso utilizaremos el dataset 'comunas' importado junto al dataset homicidios. Este dataset fue descargado del siguiente sitio web: https://data.buenosaires.gob.ar/dataset/comunas

In [18]:
#Modifico las mayúsculas por minúsculas
comunas=comunas.rename(columns={
    "COMUNAS": "COMUNA"
})

#Se realiza el merge a través de la columna en comun `Comuna` y nos va a quedar anexada la columna `Barrios`


homicidios_hechos=homicidios_hechos.merge(comunas, on="COMUNA", how="left")

#Borramos las columnas que no nos interesan del dataset, siendo estas: ID_Y, OBJETO, PERIMETRO, AREA
homicidios_hechos=homicidios_hechos.drop(columns=["ID_y", "OBJETO", "PERIMETRO", "AREA"])


Ahora vamos a agregarle a cada numero, la palabra "Comuna" usando la funcion lambda

In [19]:
# Agrego la palabra Comuna delante del dato numérico
homicidios_hechos["COMUNA"] = homicidios_hechos.apply(lambda x: "COMUNA "+str(x["COMUNA"]), axis=1)

homicidios_hechos["COMUNA"].unique()

array(['COMUNA 8', 'COMUNA 9', 'COMUNA 1', 'COMUNA 11', 'COMUNA 15',
       'COMUNA 4', 'COMUNA 7', 'COMUNA 12', 'COMUNA 3', 'COMUNA 13',
       'COMUNA 14', 'COMUNA 10', 'COMUNA 6', 'COMUNA 2', 'COMUNA 5',
       'COMUNA 0'], dtype=object)

Las colummas tienen nombres en mayúsculas y en minúsculas, los vamos a estandarizan a modo de buena práctica

In [20]:
#Cambio la primer letra a mayúscula
homicidios_hechos.columns = [x.capitalize() for x in homicidios_hechos.columns]
# Reemplazo guiones por espacios
homicidios_hechos.columns = homicidios_hechos.columns.str.replace("_", " ")
#Renombra columnas 
homicidios_hechos= homicidios_hechos.rename(columns={"N victimas": "Num víctimas",
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora",
                                            "Id x":"ID"    
                                        }) 

homicidios_hechos.columns

Index(['ID', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada', 'Barrios'],
      dtype='object')

___

### Columna dia

In [21]:
# Mostramos dia
homicidios_hechos["Día"].unique()

array([ 1,  2,  3, 10, 21, 24, 29,  8, 14, 15, 17, 28,  4, 12, 13, 19, 23,
       30, 31, 11, 20, 22, 25, 26, 16, 18, 27,  7,  9,  5,  6],
      dtype=int64)

Solo podemos ver el numero del dia, por lo que le agregamos el dia de la semana para que sea mas claro. 

In [22]:
# Mostramos dia
homicidios_hechos["Día"].unique()

array([ 1,  2,  3, 10, 21, 24, 29,  8, 14, 15, 17, 28,  4, 12, 13, 19, 23,
       30, 31, 11, 20, 22, 25, 26, 16, 18, 27,  7,  9,  5,  6],
      dtype=int64)

In [23]:
#Creo una columna nueva `Día semana` que contenga el nombre del día de la semana
homicidios_hechos['Día semana'] = homicidios_hechos['Fecha'].dt.strftime('%A')

# Mostramos dia
homicidios_hechos.sample(2)


,ID,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada,Barrios,Día semana
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,...,COMUNA 1,Point (107595.35084333 101797.50052813),-58.380486,-34.613039,MOTO-CARGAS,MOTO,CARGAS,"-58.38048577,-34.61303893",CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,Monday
690,2021-0092,1,2021-12-12,2021,12,12,06:20:00,6,AV. RIVADAVIA Y AV. PUEYRREDON,AVENIDA,...,COMUNA 3,Point (105258.35368554 102122.93231400),-58.405969,-34.610120,PEATON-AUTO,PEATON,AUTO,"-58.4059686,-34.61011987",BALVANERA - SAN CRISTOBAL,Sunday


Como los nombres se encuentran en ingles, los cambio a español

In [24]:
# Diccionario para asignar en español los nombres de los dás de la semana 
dic={  'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}

# Aplicamos el cambio
homicidios_hechos['Día semana'] =homicidios_hechos['Día semana'].replace(dic)

# Mostramos dia
homicidios_hechos.sample(3)

,ID,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada,Barrios,Día semana
269,2017-0155,1,2017-12-12,2017,12,12,21:25:41,21,AV. RIVADAVIA 6350,AVENIDA,...,COMUNA 7,Point (100674.98109200 100299.64949548),-58.455939,-34.626569,SD-MOTO,SD,MOTO,"-58.45593915,-34.62656868",FLORES - PARQUE CHACABUCO,Martes
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,COMUNA 14,Point (. .),0.000000,0.000000,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,"0.0,0.0",PALERMO,Domingo
388,2018-0114,1,2018-10-30,2018,10,30,02:15:00,2,Solis e Independencia Av.,AVENIDA,...,COMUNA 1,Point (106692.15004353 101239.02351805),-58.390329,-34.618080,MOTO-AUTO,MOTO,AUTO,"-58.39032921,-34.61807951",CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,Martes


___

## 🔁 TRANSFORM Dataset homicidios-victimas

In [25]:
print("Veamos 3 muestras del Dataset: ")
homicidios_victimas.sample(3)

Veamos 3 muestras del Dataset: 


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34,2017-03-07 00:00:00
228,2017-0096,2017-08-07,2017,8,7,PEATON,PEATON,MASCULINO,87,2017-08-07 00:00:00
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD


Veamos la descripcion estadistica con .describe()


In [26]:
homicidios_victimas.describe()

,FECHA,AAAA,MM,DD
count,717,717.000000,717.000000,717.000000
mean,2018-09-09 01:36:24.100418304,2018.174338,6.688982,15.960948
min,2016-01-01 00:00:00,2016.000000,1.000000,1.000000
25%,2017-04-01 00:00:00,2017.000000,4.000000,9.000000
50%,2018-07-07 00:00:00,2018.000000,7.000000,16.000000
75%,2019-12-18 00:00:00,2019.000000,10.000000,23.000000
max,2021-12-30 00:00:00,2021.000000,12.000000,31.000000
std,NaN,1.672655,3.575140,8.683358


### Usando una funcion personalizada data_type_check importada desde data_utils.py: 

- mostramos total de nulos, tipo de datos, dimensiones.

In [27]:
data_type_check(homicidios_victimas)


 Resumen del dataframe:

Dimensiones:  (717, 10)
               columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0             ID_hecho       100.0      0.0            0          object
1                FECHA       100.0      0.0            0  datetime64[ns]
2                 AAAA       100.0      0.0            0           int64
3                   MM       100.0      0.0            0           int64
4                   DD       100.0      0.0            0           int64
5                  ROL       100.0      0.0            0          object
6              VICTIMA       100.0      0.0            0          object
7                 SEXO       100.0      0.0            0          object
8                 EDAD       100.0      0.0            0          object
9  FECHA_FALLECIMIENTO       100.0      0.0            0          object


No encontramos datos nulos

___

### Cambiamos tipos de datos a las columnas FECHA_FALLECIMIENTO y EDAD

In [28]:
# Convertir la columna a tipo datetime
homicidios_victimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(homicidios_victimas["FECHA_FALLECIMIENTO"], format='%Y-%m-%d', errors='coerce')

# Una vez convertida a datetime, mantén solo la fecha sin la hora
homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas["FECHA_FALLECIMIENTO"].dt.date

# Reemplazar las fechas con ceros por valores vacíos para facilitar su manipulación
homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas["FECHA_FALLECIMIENTO"].replace(pd.NaT, "")

# Convertir la columna de EDAD a numérico y luego a entero
homicidios_victimas["EDAD"] = pd.to_numeric(homicidios_victimas["EDAD"], errors="coerce")
homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].astype("Int64")

# Utilizar Numpy para reemplazar los valores vacíos o sin datos por NaN y luego convertir a float para evitar errores en las estadísticas
homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].replace([" ", "SD"], np.nan)
homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].fillna(0.0)

# Verificar los datos de la columna
homicidios_victimas["EDAD"].unique()

<IntegerArray>
[19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20, 54,  0,
 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40, 87, 76, 75, 35, 80,
 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,  7, 48,  4, 82, 32, 17,
 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13, 77, 44, 31, 62, 74, 71, 11, 86,
 69, 72, 12, 63, 92, 68, 91, 64,  5, 88]
Length: 86, dtype: Int64

### Se crea la columna 'Rango etario' basada en la columna 'Edad' para mejorar el análisis de los datos


In [29]:

# Utilizamos la función `pd.cut()`, que permite agrupar valores numéricos en intervalos o categorías

# Definimos los límites de los rangos etarios y las etiquetas correspondientes
rangos_edades = [0, 20, 40, 60, 80, 100]
etiquetas_edades = ['Menos de 20', 'Entre 21 y 40', 'Entre 41 y 60', 'Entre 61 y 80', 'Más de 80']

# Utilizamos pd.cut() para crear la columna de rango etario
homicidios_victimas['Rango etario'] = pd.cut(homicidios_victimas['EDAD'], bins=rangos_edades, labels=etiquetas_edades, right=False)

# Mostramos las primeras 3 filas del DataFrame
homicidios_victimas.sample(3)


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango etario
159,2017-0018,2017-02-09,2017,2,9,CONDUCTOR,MOTO,MASCULINO,30,2017-02-09,Entre 21 y 40
75,2016-0099,2016-07-29,2016,7,29,CONDUCTOR,CARGAS,MASCULINO,45,,Entre 41 y 60
673,2021-0054,2021-06-22,2021,6,22,CONDUCTOR,MOTO,MASCULINO,41,2021-06-22,Entre 41 y 60


Estandarizamos mayusculas y minusculas

In [30]:
# Se coloca la primera en mayúscula
homicidios_victimas.columns = [x.capitalize() for x in homicidios_victimas.columns]
# Se reemplazan los guiones por espacios
homicidios_victimas.columns = homicidios_victimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
homicidios_victimas = homicidios_victimas.rename(columns={'Id hecho': 'ID',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Victima':'Víctima'})
homicidios_victimas.columns

Index(['ID', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo', 'Edad',
       'Fecha fallecimiento', 'Rango etario'],
      dtype='object')

___

Encontramos similitud en las columnas de ambos dataset, vamos a buscar el mismo id de ejemplo para notarlo mejor.


In [31]:
#Se busca un registro para comparar con el mismo registro en el otro dataset
homicidios_hechos[homicidios_hechos["ID"]=="2018-0001"]

,ID,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada,Barrios,Día semana
275,2018-0001,1,2018-01-01,2018,1,1,22:50:00,22,San Juan Av. Y Rincón,AVENIDA,...,COMUNA 3,Point (106229.70400707 100696.43939679),-58.395368,-34.622973,PEATON-MOTO,PEATON,MOTO,"-58.39536773,-34.62297346",BALVANERA - SAN CRISTOBAL,Lunes


In [32]:
homicidios_victimas[homicidios_victimas["ID"]=="2018-0001"]

,ID,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento,Rango etario
286,2018-0001,2018-01-01,2018,1,1,PEATON,PEATON,MASCULINO,83,2018-01-02,Más de 80


Vemos que se repite Fecha, año, mes, dia, victima. 

In [33]:
# Eliminamos la scolumnas repetidas
homicidios_victimas = homicidios_victimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Víctima'], axis=1)
homicidios_victimas.columns

Index(['ID', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario'], dtype='object')

___

# Unimos los dataset usando merge

In [34]:
siniestro_vial= homicidios_victimas.merge(homicidios_hechos, on="ID", how="left")
siniestro_vial.columns

Index(['ID', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa', 'Hora',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada', 'Barrios',
       'Día semana'],
      dtype='object')

In [35]:
data_type_check(siniestro_vial)


 Resumen del dataframe:

Dimensiones:  (717, 28)
                  columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0                      ID      100.00     0.00            0          object
1                     Rol      100.00     0.00            0          object
2                    Sexo      100.00     0.00            0          object
3                    Edad      100.00     0.00            0           Int64
4     Fecha fallecimiento      100.00     0.00            0          object
5            Rango etario      100.00     0.00            0        category
6            Num víctimas      100.00     0.00            0           int64
7                   Fecha      100.00     0.00            0  datetime64[ns]
8                     Año      100.00     0.00            0           int64
9                     Mes      100.00     0.00            0           int64
10                    Día      100.00     0.00            0           int64
11          Hora completa      100.00 

Encontramos algunos nulos para limpiar y cambiamos algunos tipos de datos

In [36]:
siniestro_vial ["Num víctimas"] = siniestro_vial ["Num víctimas"].fillna(0.0)
siniestro_vial ["Num víctimas"] = siniestro_vial ["Num víctimas"].astype ("Int64")

siniestro_vial ["Año"]= siniestro_vial ["Año"].astype ("Int64")
siniestro_vial ["Mes"]= siniestro_vial ["Mes"].astype ("Int64")
siniestro_vial ["Día"]= siniestro_vial ["Día"].astype ("Int64")


In [37]:
siniestro_vial.sample(2)

,ID,Rol,Sexo,Edad,Fecha fallecimiento,Rango etario,Num víctimas,Fecha,Año,Mes,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada,Barrios,Día semana
116,2016-0145,CONDUCTOR,MASCULINO,39,2016-11-07,Entre 21 y 40,1,2016-11-07,2016,11,...,COMUNA 13,Point (101657.76119068 107735.39237736),-58.445236,-34.559539,MOTO-AUTO,MOTO,AUTO,"-58.44523595,-34.55953855",BELGRANO - COLEGIALES - NUÃ‘EZ,Lunes
693,2021-0074,CONDUCTOR,MASCULINO,55,2021-09-20,Entre 41 y 60,1,2021-09-20,2021,9,...,COMUNA 12,Point (96563.66494817 108815.73881056),-58.500738,-34.549795,MULTIPLE,MOTO,MULTIPLE,"-58.5007381,-34.5497951",COGHLAN - SAAVEDRA - VILLA PUEYRREDON - VILLA ...,Lunes


Ordenamos las columnas

In [38]:
#Creo una lista con el orden 
ordenadas=["ID", "Rol", "Acusado", "Victima", "Sexo", "Edad", "Rango etario", "Año", "Mes", "Día", "Día semana", "Hora", "Hora completa", "Num víctimas", "Participantes", "Fecha fallecimiento", "Lugar del hecho", "Tipo de calle", "Calle", "Cruce", "Dirección normalizada", "Comuna", "Barrios", "Coordenada","Pos x", "Pos y"]

#Reorganizo el dataframe untilizando la lista y asignandola nuevamente aldataframe siniestros
siniestro_listo = siniestro_vial[ordenadas]
siniestro_listo.columns

Index(['ID', 'Rol', 'Acusado', 'Victima', 'Sexo', 'Edad', 'Rango etario',
       'Año', 'Mes', 'Día', 'Día semana', 'Hora', 'Hora completa',
       'Num víctimas', 'Participantes', 'Fecha fallecimiento',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Barrios', 'Coordenada', 'Pos x',
       'Pos y'],
      dtype='object')

In [39]:
data_type_check(siniestro_listo)


 Resumen del dataframe:

Dimensiones:  (717, 26)
                  columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0                      ID      100.00     0.00            0    object
1                     Rol      100.00     0.00            0    object
2                 Acusado      100.00     0.00            0    object
3                 Victima      100.00     0.00            0    object
4                    Sexo      100.00     0.00            0    object
5                    Edad      100.00     0.00            0     Int64
6            Rango etario      100.00     0.00            0  category
7                     Año      100.00     0.00            0     Int64
8                     Mes      100.00     0.00            0     Int64
9                     Día      100.00     0.00            0     Int64
10             Día semana      100.00     0.00            0    object
11                   Hora       99.86     0.14            1     Int64
12          Hora completa      100.00   

Antes de guardar hacemos una limpieza de los datos faltantes:

Eliminamos filas con valores faltantes para Acusado, Victima, Año, Mes Dia, Dia semana, Hora completa, Participantes, Lugar del hecho, Tipo de calle, Calle, Direccion normalizada, Comuna, Barrios, Coordenada x, Coordenada y, Pos x, Pos y.


# LOAD (guardamos)

Vemos como quedo nuestro ETL antes de guardar:

In [40]:
data_type_check(siniestro_listo)


 Resumen del dataframe:

Dimensiones:  (717, 26)
                  columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0                      ID      100.00     0.00            0    object
1                     Rol      100.00     0.00            0    object
2                 Acusado      100.00     0.00            0    object
3                 Victima      100.00     0.00            0    object
4                    Sexo      100.00     0.00            0    object
5                    Edad      100.00     0.00            0     Int64
6            Rango etario      100.00     0.00            0  category
7                     Año      100.00     0.00            0     Int64
8                     Mes      100.00     0.00            0     Int64
9                     Día      100.00     0.00            0     Int64
10             Día semana      100.00     0.00            0    object
11                   Hora       99.86     0.14            1     Int64
12          Hora completa      100.00   

Guardamos en csv

In [41]:
siniestro_listo.to_csv("../0 Dataset/siniestro_listo.csv", index=False)

___

## Informacion de la poblacion para KPIS

Opcional 
%pip install xlrd

In [42]:
pobla = pd.read_excel("../0 Dataset/PBP_CO_1020.xls", skiprows=2, nrows=16)
pobla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Comuna  16 non-null     object
 1   2010    16 non-null     int64 
 2   2011    16 non-null     int64 
 3   2012    16 non-null     int64 
 4   2013    16 non-null     int64 
 5   2014    16 non-null     int64 
 6   2015    16 non-null     int64 
 7   2016    16 non-null     int64 
 8   2017    16 non-null     int64 
 9   2018    16 non-null     int64 
 10  2019    16 non-null     int64 
 11  2020    16 non-null     int64 
 12  2021    16 non-null     int64 
 13  2022    16 non-null     int64 
 14  2023    16 non-null     int64 
 15  2024    16 non-null     int64 
 16  2025    16 non-null     int64 
dtypes: int64(16), object(1)
memory usage: 2.2+ KB


In [43]:
pobla.drop(0, inplace=True)

# Eliminar columnas que no nos sirven
pobla = pobla.drop(columns=[2010, 2011,2012,2013,2014,2015,2022,2023,2024,2025])
pobla.rename(columns={'Comuna':'COMUNA'}, inplace=True)
pobla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 1 to 15
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   COMUNA  15 non-null     object
 1   2016    15 non-null     int64 
 2   2017    15 non-null     int64 
 3   2018    15 non-null     int64 
 4   2019    15 non-null     int64 
 5   2020    15 non-null     int64 
 6   2021    15 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 972.0+ bytes


In [44]:
pobla.to_csv('../0 Dataset/poblacion_limpia.csv', index=False)

In [45]:
pobla.sample(2)

,COMUNA,2016,2017,2018,2019,2020,2021
15,15,182390,182409,182427,182445,182465,182485
9,9,170353,170605,170842,171062,171264,171444
